In [ ]:
import sys,os
print sys.executable

import numpy as np
import pyuvdata.utils as uvutils
import matplotlib.gridspec as gridspec

from pyuvdata import UVData
from pyuvdata.data import DATA_PATH
from astropy.time import Time
from matplotlib.pyplot import *
matplotlib.rcParams['figure.figsize'] = (16, 9)

# Read in uvfits file  
Choose data with complete filepath via uvfits_file variable

In [ ]:
# Read in uvfits data
UV = UVData()
# uvfits_file = '/Users/JBurba/Google Drive/EoR/zen.2457458.35484.xx.HH.uvcU.uvfits'
# uvfits_file = '//anaconda/envs/root_dev/lib/python2.7/site-packages/pyuvdata/data/day2_TDEM0003_10s_norx_1src_1spw.uvfits'
uvfile = os.path.join(DATA_PATH,'day2_TDEM0003_10s_norx_1src_1spw.uvfits')
UV.read_uvfits(uvfile)

# Check for the number of spectral windows
# This notebook only supports one spectral window
if UV.Nspws > 1:
    print 'Plotting currently only supported for one spectral window.'

# Options  
Defined user parameters can be used to modify the data plotted and the plot style.

### Instrument options

__ant_str__: String used for parsing antenna pair numbers and polarizations.  Basic options are 'auto', 'cross', or 'all'.  If 'all' is passed as ant_str, then ant_pairs_nums is returned as None so the data is not filtered based on antenna pair numbers in UV.select(...).  This can be passed as a combination of integer antenna numbers and polarizations [x,y,l,r].  If ant_str is passed as:
    - 9: returns all antenna pairs containing antenna number 9 (including the auto correlation) 
    - 9_10: returns only the antenna pair (9,10), i.e. ant_pairs_nums = [(9,10)] and polarizations = None 
    - (9,31)_10: returns ant_pairs_nums = [(9,10),(31,10)] and polarizations = None
    - 9_(31,10): returns ant_pairs_nums = [(9,31),(9,10)]
    - 9x_10y: returns ant_pairs_nums = [(9,10)] and polarizations = [-7]  
    - (9x,31y)_(10x,64y): returns ant_pairs_nums = [(9,10),(9,64),(31,10),(31,64)] and polarizations = [-5,-6,-7,-8]

If ant_str is passed, ant_pairs_nums and polarizations will be overwritten.

__antenna_nums__: The antennas numbers to keep in the object (antenna positions and names for the removed antennas will be retained).  This cannot be provided if antenna_names is also provided.  

__antenna_names__: The antennas names to keep in the object (antenna positions and names for the removed antennas will be retained).  This cannot be provided if antenna_nums is also provided.  

__ant_pairs_nums__: Alternate way of passing a list of tuples of antenna numbers to UV.select(...).  Follows the formatting of the output of parse_ants given __ant_str__ above, i.e. ant_pairs_nums = [(9,10),(9,31)] will only keep baselines with the matching antenna pairs (9,10) and (9,31). Ordering of the numbers within the tuple does not matter.  

__frequencies__: The frequencies to keep in the object.  

__freq_chans__: The frequency channel numbers to keep in the object.  

__times__: The times to keep in the object.  Must be passed as a list of Julian Dates.

__polarizations__: The polarizations to keep in the object in integer format.  See uvutils.polstr2num for polarization formatting.  

__blt_inds__: The baseline-time indices to keep in the object. This is not commonly used.  

__inplace__: Determines whether a new object is created when UV.select(...) is run.  Must be declared as false for this notebook to work.  


### Plotting options

__plot_type__: String that determines what aspect of the data is plotted.   Can be 'abs', 'real', 'imag', or 'phase'.  

__dmin__,__dmax__: Min and max value for plotting on the colorbar.  If left as None and no_white_space is False, each subplot will have its own min and max.  If left as None and no_white_space or force_clim is True, dmin and dmax will be set automatically and each subplot will be forced to the same min and max.  

__force_clim__: If true, all subplots are forced to the same min and max via dmin and dmax, respectively.  If dmin and dmax are left as None, the min and max values will be calculated automatically.  

__cmap__: Colormap for plotting.  Can be gist_earth, gist_heat, gist_stern, gist_yarg, hot, cool, gray, bone, spectral, copper, jet to name a few.  For a more complete list, see pylab.cm.datad.keys().

__log_scale__: If true, plot in log10.  If false, plot in linear units.  

__no_white_space__: If true, white space between subplots is removed, a master colorbar is used for all subplots, and force_clim is set to True so all subplots have the same min and max.  

__fontsize__: Sets the fontsize for each subplot's axes and colorbar labels, ticks, titles, etc.

In [ ]:
""" Instrument Options """

# Antenna string for parsing
ant_str = '(1l,2r)_(3l,6r)'

# Antennae can be passed as a list of single antenna numbers
antenna_nums = None # returns auto correlations with each antenna passed

# or as a list of antenna names
antenna_names = None

# or as a list of tuples containing the desired antennae numbers
ant_pairs_nums = None # must be a list of antenna number tuples

# Frequencies to keep in the object
frequencies = None

# Frequency channel numbers to keep in the object
freq_chans = None

# Times to keep in the object
times = None

# List of polarizations in integer format
# See uvutils.polstr2num for polarization formatting
polarizations = None

# Baseline-times to keep in the object
blt_inds = None

# If inplace = False, forces UVData.select(...) to output a new UVData object
# inplace = False is default behavior for this plotting script
inplace = False



""" Plotting Options """

# Choose plot_type, can be 'abs', 'real', 'imag', or 'phase'
plot_type = 'abs'

# Set custom dmin and dmax
dmin,dmax = None,None

# Force all subplots to use the same min and max (calculated automatically if dmin and dmax are left as None)
force_clim = True

# Choose colormap
cmap = get_cmap('jet')

# Force log10 scale for log_scale = True
log_scale = True

# Remove white space between subplots and use a master colorbar for all subplots
no_white_space = False

# Set size of subplot titles, labels, etc.
fontsize = 16

# Plotting

In [ ]:
""" Apply instrument options """

if not ant_str is None:
    ant_pairs_nums,polarizations = UV.parse_ants(ant_str)
    # If ant_str is passed, do not filter additionally based on existing antenna_nums list
    antenna_nums = None

# Apply select function with above instrument parameters
UV_selected = UV.select(antenna_nums = antenna_nums,
                        antenna_names = antenna_names,
                        ant_pairs_nums = ant_pairs_nums,
                        frequencies = frequencies,
                        freq_chans = freq_chans,
                        times = times,
                        polarizations = polarizations,
                        blt_inds = blt_inds,
                        inplace=inplace)

# Create masked array for plotting using the flag_array from UVData object
masked_data_array = np.ma.masked_array(UV_selected.data_array,UV_selected.flag_array,fill_value=np.NaN)



""" Plotting option checks """

# Check for a supported plot_type option
if not plot_type in ['abs','real','imag','phase']:
    print 'Please choose a supported plot type'
    
# If plotting phase, force linear scale
if plot_type == 'phase':
    log_scale = False

# If plotting a single baseline, set no_white_space to False for proper formatting
if UV_selected.Nbls == 1:
    no_white_space = False

# If no_white_space, automatically set force_clim=True
if no_white_space:
    force_clim = True



""" Plotting """

# Determine necessary number of rows/cols for subplots
bls = UV_selected.get_baseline_nums()
pols = UV_selected.get_pols()
nrows = int(np.sqrt(UV_selected.Nbls*UV_selected.Npols))
ncols = int(np.ceil(float(UV_selected.Nbls*UV_selected.Npols) / nrows))
nempty = nrows*ncols-UV_selected.Nbls*UV_selected.Npols

# Create figure and gridspec object
if fignum_exists(1):
    clf()
fig = figure(1,facecolor='w')
gs = gridspec.GridSpec(nrows,ncols)
plot_ind = [0,0] # [row,col] in gridspec layout
bls_counter = 0 # counter for baseline in each subplot
pol_counter = 0 # counter for polarization for each baseline

# Parameters for annotating subplots with baseline information
x_lbl_loc = 0.95 # x location of antenna pair label if no_white_space
y_lbl_loc = 0.09 # y location of antenna pair label if no_white_space

if no_white_space:
    # Add master labels for all x and y axes
    master_ax = fig.add_subplot(gs[:,:])
    master_ax.set_xlabel('Frequency [channels]',size=fontsize,labelpad=25)
    master_ax.set_ylabel('Time [integrations]',size=fontsize,labelpad=25)
    master_ax.set_xticks([])
    master_ax.set_yticks([])
    master_ax.spines['top'].set_visible(False)
    master_ax.spines['right'].set_visible(False)
    master_ax.spines['bottom'].set_visible(False)
    master_ax.spines['left'].set_visible(False)

# Assign range for colorbar range if dmin and dmax not specified
if dmin is None and dmax is None:
    if log_scale:
        if plot_type == 'abs':
            dmin = np.ma.min(np.log10(np.abs(masked_data_array)))
            dmax = np.ma.max(np.log10(np.abs(masked_data_array)))
        elif plot_type == 'real':
            dmin = np.ma.min(np.log10(np.real(masked_data_array)))
            dmax = np.ma.max(np.log10(np.real(masked_data_array)))
        elif plot_type == 'imag':
            dmin = np.ma.min(np.log10(np.imag(masked_data_array)))
            dmax = np.ma.max(np.log10(np.imag(masked_data_array)))
        else:
            print 'Cannot have log_scale with plot_type= \'phase\''
    else:
        if plot_type == 'abs':
            dmin = np.ma.min(np.abs(masked_data_array))
            dmax = np.ma.max(np.abs(masked_data_array))
        elif plot_type == 'real':
            dmin = np.ma.min(np.real(masked_data_array))
            dmax = np.ma.max(np.real(masked_data_array))
        elif plot_type == 'imag':
            dmin = np.ma.min(np.imag(masked_data_array))
            dmax = np.ma.max(np.imag(masked_data_array))
        else:
            dmin = -np.pi
            dmax = np.pi

# Iterate through gridspec subplot array
while plot_ind[0] < nrows and bls_counter < UV_selected.Nbls:
    while plot_ind[1] < ncols and bls_counter < UV_selected.Nbls:
        # Get indices for given baseline
        bl = bls[bls_counter]
        bls_inds = np.where(UV_selected.baseline_array == bl)[0]
        
        # Plot each polarization for a given baseline
        for pol in pols:
            ax = fig.add_subplot(gs[plot_ind[0],plot_ind[1]])
            pol_ind = np.where(UV_selected.polarization_array == uvutils.polstr2num(pol))[0]
            if plot_type == 'abs':
                if log_scale:
                    im = ax.imshow(np.log10(np.abs(masked_data_array[bls_inds,0,:,pol_ind])),
                                   origin='upper',aspect='auto', interpolation='none',
                                   cmap=cmap) #,vmin=dmin,vmax=dmax)
                else:
                    im = ax.imshow(np.abs(masked_data_array[bls_inds,0,:,pol_ind]),
                                   origin='upper',aspect='auto', interpolation='none',
                                   cmap=cmap) #,vmin=dmin,vmax=dmax)
            elif plot_type == 'real':
                if log_scale:
                    im = ax.imshow(np.log10(np.real(masked_data_array[bls_inds,0,:,pol_ind])),
                                   origin='upper',aspect='auto', interpolation='none',
                                   cmap=cmap) #,vmin=dmin,vmax=dmax)
                else:
                    im = ax.imshow(np.real(masked_data_array[bls_inds,0,:,pol_ind]),
                                   origin='upper',aspect='auto', interpolation='none',
                                   cmap=cmap) #,vmin=dmin,vmax=dmax)
            elif plot_type == 'imag':
                if log_scale:
                    im = ax.imshow(np.log10(np.imag(masked_data_array[bls_inds,0,:,pol_ind])),
                                   origin='upper',aspect='auto', interpolation='none',
                                   cmap=cmap) #,vmin=dmin,vmax=dmax)
                else:
                    im = ax.imshow(np.imag(masked_data_array[bls_inds,0,:,pol_ind]),
                                   origin='upper',aspect='auto', interpolation='none',
                                   cmap=cmap) #,vmin=dmin,vmax=dmax)
            else:
                im = ax.imshow(np.angle(masked_data_array[bls_inds,0,:,pol_ind]),
                               origin='upper',aspect='auto', interpolation='none',
                               cmap=cmap) #,vmin=dmin,vmax=dmax)

            if force_clim:
                # Force plots to same clim
                im.set_clim(dmin,dmax)

            # Make label for subplot
            ant_pair = str(UV_selected.baseline_to_antnums(bl)).strip('()')
            ant_pair = ant_pair.split(',')
            ax_label = ant_pair[0]+pol[0]+','+ant_pair[1]+pol[1]
            
            if no_white_space:
                # remove axis labels from interior plots
                if nrows*ncols == UV_selected.Nbls*UV_selected.Npols:
                    if not plot_ind[0] == nrows-1:
                        ax.set_xticks([])
                else:
                    if plot_ind[0] < nrows-2:
                        ax.set_xticks([])
                    elif plot_ind[0] == nrows-2 and plot_ind[1] <= ncols-nempty-1:
                        ax.set_xticks([])

                if not plot_ind[1] == 0:
                    ax.set_yticks([])

                # Add antenna numbers to subplot
                ax.text(x_lbl_loc*UV_selected.Nfreqs,y_lbl_loc*UV_selected.Ntimes,
                        ax_label,ha='right',size=fontsize)  
            else:
                # Add title and axis labels to each subplot
                ax.set_title(ax_label,size=fontsize)
                ax.set_ylabel('Time [integrations]',size=fontsize)
                ax.set_xlabel('Frequency [channels]',size=fontsize)
                ax.tick_params(axis='both',labelsize=fontsize)

                # Add colorbars to subplots
                cb = fig.colorbar(im,ax=ax,format='%.1f')
                cb.ax.tick_params(labelsize=fontsize)
        
            # Update column counter
            plot_ind[1] += 1
        bls_counter += 1
        
    # Reset column counter and update row counter
    plot_ind[1] = 0
    plot_ind[0] +=1

# Update plot dimensions to add master colorbar
gs.tight_layout(fig)

if no_white_space:
    # Append master colorbar
    gs.update(wspace=0.0,hspace=0.0,right=0.85)
    #gs2 = gridspec.GridSpec(1,3)
    #gs2.update(left=0.9,right=1.0,wspace=0.0,hspace=0.0)
    #cbar_ax = fig.add_subplot(gs2[1])
    cbar_ax = fig.add_axes([0.875, 0.15, 0.02, 0.75])
    #cb_ticks = np.linspace(dmin,dmax,10).astype('int')
    cb = fig.colorbar(im, cax=cbar_ax)#,ticks=cb_ticks)
    #cb.ax.set_yticklabels(cb_ticks.astype('str'))
    cb.set_clim(dmin,dmax)
    
    if log_scale:
        cb.set_label('Log Visibilities',size=fontsize,labelpad=10)
    
    cb.ax.tick_params(labelsize=fontsize)
    
show()

In [ ]:
if fignum_exists(1):
    print 'True'





  
  
  
  









------------

# Old plotting code

## Iteration 1

## Iteration 2

# Iteration 3